In [6]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import os
import sys 
from skimage import data, io, filters
import configparser
import pandas as pd 
import numpy as np
import torch
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

from matplotlib import cycler
from os.path import join as oj
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader
import pickle as pkl
from sklearn.preprocessing import StandardScaler,MinMaxScaler
sys.path.insert(0, "../src")

from torch import optim
from torch.utils.data import DataLoader, TensorDataset
import my_data
from utils.dice_score import dice_loss
from evaluate import evaluate
from unet import UNet
from predict import predict_img
import seaborn as sns
import pickle as pkl
from evaluate import aq_cost_function
pd.set_option('display.float_format', lambda x: '%.2f' % x)
cuda = False # torch.cuda.is_available()
device = torch.device("cuda:0" if cuda else "cpu")
config = configparser.ConfigParser()
config.read('../config.ini');
data_path =config['DATASET']['data_path'] 

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
img_list = my_data.load_data(data_path)


In [8]:
in_arr = np.asarray([x[0] for x in img_list])


In [9]:
in_arr.shape

(6, 2048, 2048)

In [10]:
img_list[0][0]

array([[106, 106, 106, ..., 106, 106, 106],
       [106, 106, 106, ..., 106, 106, 106],
       [106, 106, 106, ..., 106, 106, 106],
       ...,
       [106, 106, 106, ..., 106, 106, 106],
       [106, 106, 106, ..., 106, 106, 106],
       [106, 106, 106, ..., 106, 106, 106]], dtype=uint8)

In [64]:
new_data_path = oj(os.path.split(data_path)[0], 'lno_full')

In [14]:
test = my_data.load_data(oj(os.path.split(data_path)[0], 'lno_full'))


In [26]:
from tqdm import tqdm_notebook

In [39]:
idx_list = []
for i in tqdm_notebook(range(len(test))):
    if np.any(np.all(test[i][None,:] == in_arr, axis=(1,2)) ):
        idx_list.append((i, np.argmax(np.all(test[i][None,:] == in_arr, axis=(1,2)) )))

C:\Users\lauri\AppData\Local\Temp\ipykernel_6360\1403176694.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm_notebook(range(len(test))):


  0%|          | 0/2048 [00:00<?, ?it/s]

In [40]:
idx_list

[(229, 0), (519, 1), (769, 2), (1129, 3), (1529, 4), (1839, 5)]

In [61]:
img_list = []
for (whole_idx, _) in idx_list:
    img_list.append(test[whole_idx-2:whole_idx+3])

In [62]:
import pickle as pkl

In [63]:
data_path

'C:\\Users\\lauri\\Documents\\GitHub\\segmenter\\data\\lno'

In [66]:
with open(oj(new_data_path, 'extended_LNO.pkl'), 'wb') as f:
    pkl.dump(np.asarray(img_list), f)